In [60]:
%load_ext autoreload
%autoreload 2
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lark import Lark
import time 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
def sklearn_to_df(sklearn_dataset):
    """ 
    Convert sklearn dataset to a dataframe, the class-label is renamed to "target"
    """
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

In [62]:
target = "target"
dataset = datasets.load_iris()
dataset[target] = np.where(dataset[target]==2, 0, dataset[target])

In [63]:
# get df
data = sklearn_to_df(dataset)

# get X,y
X = data.drop(['target'], axis=1)
y = data['target']

X.columns = ['sepal length', 'sepal width', 'petal length', 'petal width']


# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 70% training and 30% test

In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

clf_rf=RandomForestClassifier(n_estimators=100)
clf_mlp = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

clf_rf.fit(X_train,y_train)
clf_mlp.fit(X_train,y_train)

y_pred=clf_mlp.predict(X_test)

In [65]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [66]:
print(clf_rf.predict([[3, 5, 4, 2]]))
print(clf_mlp.predict([[3, 5, 4, 2]]))

[0]
[0]


In [67]:
# create a random generator
import random
def random_generator(X):
    num_attributes = len(X.columns)
    x=[]
    for i in range(num_attributes):
        x.append(random.uniform(X[X.columns[i]].min(),X[X.columns[i]].max()))
    return x



In [68]:
for i in range(5):
    x = random_generator(X)
    print(x)
    print(clf_rf.predict([x]))
    print(clf_mlp.predict([x]))

[7.278866599069033, 2.6018796609180237, 4.1881701435908445, 0.4453968444750017]
[0]
[1]
[5.550811372493768, 3.5010769654736995, 2.8244705860256785, 0.7154328970940397]
[1]
[0]
[5.1928241823214405, 3.6097973368097476, 1.4164975931638688, 0.7047695757419707]
[0]
[0]
[7.778443081088669, 3.7089819330652967, 2.266293241186951, 2.3817767255480584]
[0]
[0]
[6.259626150227698, 2.5034751671186077, 6.321623029918204, 1.0926429300918863]
[0]
[0]


In [69]:
from blackbox import BlackBox
bb = BlackBox(clf_mlp, clf_mlp.predict)

In [70]:
bb.classify_example([3, 5, 4, 2])

0

In [71]:
from sygus_if import SyGuS_IF
X = [
    [1,2,1,1],
    [-1.1,1,1,42]
]
y = [
    1,
    0
]

In [72]:
sgf = SyGuS_IF()
# sgf.fit(X,y)
sgf.fit(X_train,y_train)
print(sgf.synthesized_function)

(define-fun func ((sepal_length Real) (sepal_width Real) (petal_length Real) (petal_width Real)) Real (ite (>= petal_length 2) (ite (> sepal_width 2) (ite (not (= petal_width 2)) (ite (> petal_length 1) (ite (<= sepal_width 4) (ite (< sepal_width 4) (ite (>= sepal_width (/ 5 2)) (ite (> sepal_width (/ 5 2)) (ite (>= petal_length (/ 4 3)) (ite (>= petal_length (/ 3 2)) (ite (> petal_length (/ 3 2)) (ite (<= sepal_width 3) (ite (<= petal_width 2) (ite (<= petal_length 6) (ite (<= petal_length 5) (ite (< petal_length 5) (ite (< sepal_width 3) (ite (<= sepal_length 6) (ite (< sepal_length 6) (ite (<= petal_length 4) (ite (< petal_length 4) (ite (> petal_width 1) 1 petal_width) (* (/ 1 4) petal_length)) (ite (<= petal_width 1) petal_width (ite (>= petal_length 5) 0 1))) 1) (ite (<= petal_length 4) (* (/ 1 4) petal_length) (ite (>= petal_width (/ 3 2)) (ite (= petal_width (/ 3 2)) (- petal_width (/ 1 2)) 0) 1))) (ite (> sepal_length 5) (ite (<= sepal_length 6) (ite (< sepal_length 6) (- sepa

In [73]:
start_ = time.time()



y_pred_test = sgf.predict_z3(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
y_pred_train = sgf.predict_z3(X_train)
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

print(time.time() - start_)

Accuracy: 1.0
Accuracy: 1.0
2.282406806945801


In [74]:


start_ = time.time()

y_pred_test = sgf.predict(X_test, y_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
y_pred_train = sgf.predict(X_train, y_train)
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

print(time.time() - start_)

Accuracy: 1.0
Accuracy: 1.0
3.8154234886169434
